In [24]:
import pandas as pd
import psycopg2
from sshtunnel import SSHTunnelForwarder

In [71]:
data = pd.read_csv('users.csv')
data = data.drop('Label',1)
data = data.drop('timeset',1)
data = data.rename(columns = {'Id':'user_id'})
topic = pd.read_csv('topics.csv')

In [65]:
def fetchTopic(topic):
    idtweet_topic_pol = "SELECT twitter_id FROM public.tweet JOIN public.tweet_topic ON id_tweet = id WHERE id_topic = "+topic
    replies = "SELECT id, user_id FROM public.tweet JOIN ("+ idtweet_topic_pol +") as Pol ON tweet.in_reply_twitter_id = Pol.twitter_id"
    replies_sentiment = "SELECT id, user_id, sentiment FROM tweet_sentiment JOIN ("+ replies +") as Rep ON id_tweet = id;"
    try:
        with SSHTunnelForwarder(
            ('161.35.123.231', 22),
            ssh_username="postgres",
            ssh_password="dbConn2021!", 
            remote_bind_address=('localhost', 5432)) as server:

            print("server connected")

            keepalive_kwargs = {
                "keepalives": 1,
                "keepalives_idle": 60,
                "keepalives_interval": 10,
                "keepalives_count": 5
            }
            params = {
                'database': 'tweetproject',
                'user': 'postgres',
                'password': 'padova2021',
                'host': server.local_bind_host,
                'port': server.local_bind_port,
                **keepalive_kwargs
                }

            conn = psycopg2.connect(**params)
            curs = conn.cursor()
            print("database connected")
            curs.execute(replies_sentiment)
            output = pd.DataFrame.from_records(curs, columns = ['id','user_id','sentiment'])
            print("query done")
            return output

    except (Exception) as error:
            print("Connection Failed")
            print(error)

In [67]:
replies = {}
for tp_id in topic['id']:
    replies[tp_id] = fetchTopic(str(tp_id))

server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done
server connected
database connected
query done


In [96]:
mod_classes = data.groupby('modularity_class')
for tp_id,tp_label in zip(topic['id'],topic['label']):
    out = pd.DataFrame(columns = ('modularity_class','sentiment'))
    for community in mod_classes:
        temp = community[1].merge(replies[tp_id], how = 'inner', on = 'user_id')
        out = pd.concat([out,temp[['modularity_class', 'sentiment']]])
    out[['modularity_class', 'sentiment']].to_csv('data/'+tp_label+'.csv')